In [1]:
import math
import numpy as np
from joblib import load
from sklearn.ensemble import GradientBoostingClassifier


# Loading in final features test flows dicts
#
# Returns: all unknown test flows dict, mirror test flows dict, known test flows dict
def load_final_test_dicts(N):
    if N == 100:
        mirror_test_flows = load(f"../Feature-Vectors/train-test-flows/{N}-p-subflows/Final-80-Mirror-TEST-Flows-Dict")
        unknown_test_flows = load(f"../Feature-Vectors/train-test-flows/{N}-p-subflows/Final-80-Random-Unknown-TEST-Flows-Dict")
        known_test_flows = load(f"../Feature-Vectors/train-test-flows/{N}-p-subflows/Final-80-Random-Known-TEST-Flows-Dict")
    else:
        mirror_test_flows = load(f"../Feature-Vectors/train-test-flows/{N}-p-subflows/Final-80-Mirror-TEST-Flows-Dict")
        unknown_test_flows = load(f"../Feature-Vectors/train-test-flows/{N}-p-subflows/Final-80-Unknown-TEST-Flows-Dict")
        known_test_flows = load(f"../Feature-Vectors/train-test-flows/{N}-p-subflows/Final-80-Known-TEST-Flows-Dict")
    return unknown_test_flows, mirror_test_flows, known_test_flows

# Same as above but naive features
def load_naive_test_dicts(N):
    unknown_test_flows = load(f"../Feature-Vectors/train-test-flows/Basic-Features/{N}-p-subflows/Unknown-TEST-Flows-Dict")
    mirror_test_flows = load(f"../Feature-Vectors/train-test-flows/Basic-Features/{N}-p-subflows/Mirror-TEST-Flows-Dict")
    known_test_flows = load(f"../Feature-Vectors/train-test-flows/Basic-Features/{N}-p-subflows/Known-TEST-Flows-Dict")
    return unknown_test_flows, mirror_test_flows, known_test_flows


# Create dict of partial flows given a percentage of subflows to include and flow dict
# Partial flows must have at least 15 subflows to be included
def get_partial_flows(flow_dict, percentage):
    partial_flows = {}
    for flow in flow_dict:
        subflows = flow_dict[flow]
        portion = math.ceil(percentage * len(subflows))
        if len(subflows) < 15:
            continue
        partial_flows[flow] = subflows[:portion]            
    return partial_flows


# Classify with both confidences, get uncertain percentage
# NOTE: uncertain count if maj=True is just count of flows classified with majority
def classify_confidences(label, known_con, unknown_con, con_thresh, maj=False):
    correct = 0
    uncertain = 0
    # Find the likelihood ratio that the confidence requires
    needed_ratio = con_thresh / (1-con_thresh)
    needed_ratio = math.log(needed_ratio)
    assert len(known_con) == len(unknown_con)
    for con_ind in range(len(unknown_con)):
        k_con = known_con[con_ind]
        u_con = unknown_con[con_ind]
        assert not (k_con > needed_ratio and u_con > needed_ratio)
        if k_con > needed_ratio:
            if label == 1: correct += 1;
        elif u_con > needed_ratio:
            if label == 0: correct += 1;
        else:
            uncertain += 1
            # Classify by whichever confidence is larger if majority is true
            if maj:
                classification = 1 if k_con > u_con else 0
                if classification == label: correct += 1;
    uncertain = uncertain/len(known_con)
    return correct, uncertain, correct/len(known_con)


# TODO: ONLY GET CONFIDENCE IF FLOW HAS > 15 SUBFLOWS
# Get known and unknown confidences for all flows in given flow dictionary, 
# Given a trained GBDT model and bins/likelihoods
# Returned "flows" dict IS NOT USED DON'T USE IT (doesn't reflect thrown out < 15 subflow flows)
def get_flow_confidences(flow_dict, known_bin_likelihoods, unknown_bin_likelihoods, gbdt):
    flows = list(flow_dict.keys())
    known_confidences = []
    unknown_confidences = []
    for flow in flow_dict:
        subflows = flow_dict[flow]
        if len(subflows) < 15:
            continue
        # Get predictions on the flow's subflows
        predictions = gbdt.predict(subflows)
        # Calculate sum of logs of known and unknown likelihoods for the flow's subflows
        known_ll_sum = 0
        unknown_ll_sum = 0
        for p in predictions:
            # DEBUG/TESTING CODE
#             print(f"Prediction: {p}")
            if p == 1:
                known_l = known_bin_likelihoods[1] 
                unknown_l = known_bin_likelihoods[0] 
#                 print(f"Known L: {known_l} \t Unknown L: {unknown_l}")
            elif p == 0:
                known_l = unknown_bin_likelihoods[1] 
                unknown_l = unknown_bin_likelihoods[0] 
#                 print(f"Known L: {known_l} \t Unknown L: {unknown_l}")
            known_ll_sum += known_l
            unknown_ll_sum += unknown_l
        # Known confidence is sum of known lls - sum of unknown lls (vice versa for unknown)
        # Which are just the likelihood ratios in the log space
#         print(f"Known Confidence Sum: {known_ll_sum} \t Unknown Confidence Sum: {unknown_ll_sum}")
        known_confidence = known_ll_sum - unknown_ll_sum
        unknown_confidence = unknown_ll_sum - known_ll_sum
        known_confidences.append(known_confidence)
        unknown_confidences.append(unknown_confidence)
    return flows, known_confidences, unknown_confidences

### Naive Features

In [2]:
############################# 25 PACKET SUBFLOWS - GBDT 
N = 25
unknown_test_flows, mirror_test_flows, known_test_flows = load_naive_test_dicts(N) 
con_thresh = 0.95

# MIRROR UNKNOWN
print("MIRROR UNKNOWN ############################################")
name = f'{N}_M_n'
# Load models
mirror_known_bins = np.load(f"Models/{name}_Known_Bin_Ls.npy")
mirror_unknown_bins = np.load(f"Models/{name}_Unknown_Bin_Ls.npy")
gbdt = load(f'Models/{name}_GBDT')

# 25, 50, 75% of flows
p = [0.25, 0.5, 0.75]
for percentage in p:
    p_unknown_flows = get_partial_flows(mirror_test_flows, percentage)
    p_known_flows = get_partial_flows(known_test_flows, percentage)
    print(f"Flow Percentage: {percentage}")
    print(f"Full Unknown Flows Length: {len(mirror_test_flows)} \t Full Known Flows Length: {len(known_test_flows)}")
    print(f"Partial Unknown Flows Length: {len(p_unknown_flows)} \t Partial Known Flows Length: {len(p_known_flows)}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_known_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
    print(f"Known Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Known Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_unknown_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
    print(f"Unknown Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Unknown Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}\n")

# Full flows
flows, known_confidences, unknown_confidences = get_flow_confidences(known_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
print(f"Known Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Known Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
flows, known_confidences, unknown_confidences = get_flow_confidences(mirror_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
print(f"Unknown Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Unknown Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")


# ALL UNKNOWN
print("\n\nALL UNKNOWN #############################################")
name = f'{N}_U_n'
# Load models
mirror_known_bins = np.load(f"Models/{name}_Known_Bin_Ls.npy")
mirror_unknown_bins = np.load(f"Models/{name}_Unknown_Bin_Ls.npy")
gbdt = load(f'Models/{name}_GBDT')

# 25, 50, 75% of flows
p = [0.25, 0.5, 0.75]
for percentage in p:
    p_unknown_flows = get_partial_flows(unknown_test_flows, percentage)
    p_known_flows = get_partial_flows(known_test_flows, percentage)
    print(f"Flow Percentage: {percentage}")
    print(f"Full Unknown Flows Length: {len(unknown_test_flows)} \t Full Known Flows Length: {len(known_test_flows)}")
    print(f"Partial Unknown Flows Length: {len(p_unknown_flows)} \t Partial Known Flows Length: {len(p_known_flows)}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_known_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
    print(f"Known Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Known Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_unknown_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
    print(f"Unknown Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Unknown Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}\n")

# Full flows
flows, known_confidences, unknown_confidences = get_flow_confidences(known_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
print(f"Known Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Known Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
flows, known_confidences, unknown_confidences = get_flow_confidences(unknown_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
print(f"Unknown Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Unknown Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")

MIRROR UNKNOWN ############################################
Flow Percentage: 0.25
Full Unknown Flows Length: 348 	 Full Known Flows Length: 41
Partial Unknown Flows Length: 338 	 Partial Known Flows Length: 40
Known Strict 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 39
Known Maj. 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 39
Unknown Strict 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 205
Unknown Maj. 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 205

Flow Percentage: 0.5
Full Unknown Flows Length: 348 	 Full Known Flows Length: 41
Partial Unknown Flows Length: 338 	 Partial Known Flows Length: 40
Known Strict 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 40
Known Maj. 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 40
Unknown Strict 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 309
Unknown Maj. 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 309

Flow Percentage: 0.75
Full Unknown Flows Length: 348 	 Full Known Flows Length: 41
Partial Unknown Flows Length

In [3]:
############################# 100 PACKET SUBFLOWS - GBDT 
N = 100
unknown_test_flows, mirror_test_flows, known_test_flows = load_naive_test_dicts(N) 
con_thresh = 0.95

# MIRROR UNKNOWN
print("MIRROR UNKNOWN ############################################")
name = f'{N}_M_n'
# Load models
mirror_known_bins = np.load(f"Models/{name}_Known_Bin_Ls.npy")
mirror_unknown_bins = np.load(f"Models/{name}_Unknown_Bin_Ls.npy")
gbdt = load(f'Models/{name}_GBDT')

# 25, 50, 75% of flows
p = [0.25, 0.5, 0.75]
for percentage in p:
    p_unknown_flows = get_partial_flows(mirror_test_flows, percentage)
    p_known_flows = get_partial_flows(known_test_flows, percentage)
    print(f"Flow Percentage: {percentage}")
    print(f"Full Unknown Flows Length: {len(mirror_test_flows)} \t Full Known Flows Length: {len(known_test_flows)}")
    print(f"Partial Unknown Flows Length: {len(p_unknown_flows)} \t Partial Known Flows Length: {len(p_known_flows)}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_known_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
    print(f"Known Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Known Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_unknown_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
    print(f"Unknown Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Unknown Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}\n")

# Full flows
flows, known_confidences, unknown_confidences = get_flow_confidences(known_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
print(f"Known Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Known Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
flows, known_confidences, unknown_confidences = get_flow_confidences(mirror_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
print(f"Unknown Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Unknown Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")


# ALL UNKNOWN
print("\n\nALL UNKNOWN #############################################")
name = f'{N}_U_n'
# Load models
mirror_known_bins = np.load(f"Models/{name}_Known_Bin_Ls.npy")
mirror_unknown_bins = np.load(f"Models/{name}_Unknown_Bin_Ls.npy")
gbdt = load(f'Models/{name}_GBDT')

# 25, 50, 75% of flows
p = [0.25, 0.5, 0.75]
for percentage in p:
    p_unknown_flows = get_partial_flows(unknown_test_flows, percentage)
    p_known_flows = get_partial_flows(known_test_flows, percentage)
    print(f"Flow Percentage: {percentage}")
    print(f"Full Unknown Flows Length: {len(unknown_test_flows)} \t Full Known Flows Length: {len(known_test_flows)}")
    print(f"Partial Unknown Flows Length: {len(p_unknown_flows)} \t Partial Known Flows Length: {len(p_known_flows)}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_known_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
    print(f"Known Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Known Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_unknown_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
    print(f"Unknown Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Unknown Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}\n")

# Full flows
flows, known_confidences, unknown_confidences = get_flow_confidences(known_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
print(f"Known Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Known Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
flows, known_confidences, unknown_confidences = get_flow_confidences(unknown_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
print(f"Unknown Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Unknown Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")

MIRROR UNKNOWN ############################################
Flow Percentage: 0.25
Full Unknown Flows Length: 379 	 Full Known Flows Length: 39
Partial Unknown Flows Length: 215 	 Partial Known Flows Length: 36
Known Strict 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 35
Known Maj. 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 35
Unknown Strict 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 54
Unknown Maj. 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 54

Flow Percentage: 0.5
Full Unknown Flows Length: 379 	 Full Known Flows Length: 39
Partial Unknown Flows Length: 215 	 Partial Known Flows Length: 36
Known Strict 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 35
Known Maj. 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 35
Unknown Strict 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 120
Unknown Maj. 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 120

Flow Percentage: 0.75
Full Unknown Flows Length: 379 	 Full Known Flows Length: 39
Partial Unknown Flows Length: 

In [6]:
############################# 1000 PACKET SUBFLOWS - GBDT 
N = 1000
unknown_test_flows, mirror_test_flows, known_test_flows = load_naive_test_dicts(N) 
con_thresh = 0.95

# MIRROR UNKNOWN
print("MIRROR UNKNOWN ############################################")
name = f'{N}_M_n'
# Load models
mirror_known_bins = np.load(f"Models/{name}_Known_Bin_Ls.npy")
mirror_unknown_bins = np.load(f"Models/{name}_Unknown_Bin_Ls.npy")
gbdt = load(f'Models/{name}_GBDT')

# 25, 50, 75% of flows
p = [0.25, 0.5, 0.75]
for percentage in p:
    p_unknown_flows = get_partial_flows(mirror_test_flows, percentage)
    p_known_flows = get_partial_flows(known_test_flows, percentage)
    print(f"Flow Percentage: {percentage}")
    print(f"Full Unknown Flows Length: {len(mirror_test_flows)} \t Full Known Flows Length: {len(known_test_flows)}")
    print(f"Partial Unknown Flows Length: {len(p_unknown_flows)} \t Partial Known Flows Length: {len(p_known_flows)}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_known_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
    print(f"Known Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Known Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_unknown_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
    print(f"Unknown Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Unknown Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}\n")

# Full flows
flows, known_confidences, unknown_confidences = get_flow_confidences(known_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
print(f"Known Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Known Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
flows, known_confidences, unknown_confidences = get_flow_confidences(mirror_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
print(f"Unknown Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Unknown Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")


# ALL UNKNOWN
print("\n\nALL UNKNOWN #############################################")
name = f'{N}_U_n'
# Load models
mirror_known_bins = np.load(f"Models/{name}_Known_Bin_Ls.npy")
mirror_unknown_bins = np.load(f"Models/{name}_Unknown_Bin_Ls.npy")
gbdt = load(f'Models/{name}_GBDT')

# 25, 50, 75% of flows
p = [0.25, 0.5, 0.75]
for percentage in p:
    p_unknown_flows = get_partial_flows(unknown_test_flows, percentage)
    p_known_flows = get_partial_flows(known_test_flows, percentage)
    print(f"Flow Percentage: {percentage}")
    print(f"Full Unknown Flows Length: {len(unknown_test_flows)} \t Full Known Flows Length: {len(known_test_flows)}")
    print(f"Partial Unknown Flows Length: {len(p_unknown_flows)} \t Partial Known Flows Length: {len(p_known_flows)}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_known_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
    print(f"Known Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Known Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_unknown_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
    print(f"Unknown Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Unknown Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}\n")

# Full flows
flows, known_confidences, unknown_confidences = get_flow_confidences(known_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
print(f"Known Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Known Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
flows, known_confidences, unknown_confidences = get_flow_confidences(unknown_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
print(f"Unknown Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Unknown Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")

MIRROR UNKNOWN ############################################
Flow Percentage: 0.25
Full Unknown Flows Length: 417 	 Full Known Flows Length: 22
Partial Unknown Flows Length: 18 	 Partial Known Flows Length: 21
Known Strict 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 20
Known Maj. 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 20
Unknown Strict 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 7
Unknown Maj. 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 7

Flow Percentage: 0.5
Full Unknown Flows Length: 417 	 Full Known Flows Length: 22
Partial Unknown Flows Length: 18 	 Partial Known Flows Length: 21
Known Strict 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 20
Known Maj. 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 20
Unknown Strict 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 9
Unknown Maj. 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 9

Flow Percentage: 0.75
Full Unknown Flows Length: 417 	 Full Known Flows Length: 22
Partial Unknown Flows Length: 18 	 Par

In [5]:
############################# 100 PACKET SUBFLOWS - TREE 
N = 100
unknown_test_flows, mirror_test_flows, known_test_flows = load_naive_test_dicts(N) 
con_thresh = 0.95

# MIRROR UNKNOWN
print("MIRROR UNKNOWN ############################################")
name = f'{N}_M_Tree'
# Load models
mirror_known_bins = np.load(f"Models/{name}_Known_Bin_Ls.npy")
mirror_unknown_bins = np.load(f"Models/{name}_Unknown_Bin_Ls.npy")
gbdt = load(f'Models/{name}')

# 25, 50, 75% of flows
p = [0.25, 0.5, 0.75]
for percentage in p:
    p_unknown_flows = get_partial_flows(mirror_test_flows, percentage)
    p_known_flows = get_partial_flows(known_test_flows, percentage)
    print(f"Flow Percentage: {percentage}")
    print(f"Full Unknown Flows Length: {len(mirror_test_flows)} \t Full Known Flows Length: {len(known_test_flows)}")
    print(f"Partial Unknown Flows Length: {len(p_unknown_flows)} \t Partial Known Flows Length: {len(p_known_flows)}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_known_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
    print(f"Known Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Known Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_unknown_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
    print(f"Unknown Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Unknown Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}\n")

# Full flows
flows, known_confidences, unknown_confidences = get_flow_confidences(known_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
print(f"Known Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Known Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
flows, known_confidences, unknown_confidences = get_flow_confidences(mirror_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
print(f"Unknown Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Unknown Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")


# ALL UNKNOWN
print("\n\nALL UNKNOWN #############################################")
name = f'{N}_U_Tree'
# Load models
mirror_known_bins = np.load(f"Models/{name}_Known_Bin_Ls.npy")
mirror_unknown_bins = np.load(f"Models/{name}_Unknown_Bin_Ls.npy")
gbdt = load(f'Models/{name}')

# 25, 50, 75% of flows
p = [0.25, 0.5, 0.75]
for percentage in p:
    p_unknown_flows = get_partial_flows(unknown_test_flows, percentage)
    p_known_flows = get_partial_flows(known_test_flows, percentage)
    print(f"Flow Percentage: {percentage}")
    print(f"Full Unknown Flows Length: {len(unknown_test_flows)} \t Full Known Flows Length: {len(known_test_flows)}")
    print(f"Partial Unknown Flows Length: {len(p_unknown_flows)} \t Partial Known Flows Length: {len(p_known_flows)}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_known_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
    print(f"Known Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Known Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_unknown_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
    print(f"Unknown Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Unknown Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}\n")

# Full flows
flows, known_confidences, unknown_confidences = get_flow_confidences(known_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
print(f"Known Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Known Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
flows, known_confidences, unknown_confidences = get_flow_confidences(unknown_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
print(f"Unknown Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Unknown Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")

MIRROR UNKNOWN ############################################
Flow Percentage: 0.25
Full Unknown Flows Length: 379 	 Full Known Flows Length: 39
Partial Unknown Flows Length: 215 	 Partial Known Flows Length: 36
Known Strict 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 36
Known Maj. 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 36
Unknown Strict 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 215
Unknown Maj. 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 215

Flow Percentage: 0.5
Full Unknown Flows Length: 379 	 Full Known Flows Length: 39
Partial Unknown Flows Length: 215 	 Partial Known Flows Length: 36
Known Strict 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 36
Known Maj. 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 36
Unknown Strict 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 215
Unknown Maj. 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 215

Flow Percentage: 0.75
Full Unknown Flows Length: 379 	 Full Known Flows Length: 39
Partial Unknown Flows Length

In [6]:
############################# 1000 PACKET SUBFLOWS - TREE 
N = 1000
unknown_test_flows, mirror_test_flows, known_test_flows = load_naive_test_dicts(N) 
con_thresh = 0.95

# MIRROR UNKNOWN
print("MIRROR UNKNOWN ############################################")
name = f'{N}_M_Tree'
# Load models
mirror_known_bins = np.load(f"Models/{name}_Known_Bin_Ls.npy")
mirror_unknown_bins = np.load(f"Models/{name}_Unknown_Bin_Ls.npy")
gbdt = load(f'Models/{name}')

# 25, 50, 75% of flows
p = [0.25, 0.5, 0.75]
for percentage in p:
    p_unknown_flows = get_partial_flows(mirror_test_flows, percentage)
    p_known_flows = get_partial_flows(known_test_flows, percentage)
    print(f"Flow Percentage: {percentage}")
    print(f"Full Unknown Flows Length: {len(mirror_test_flows)} \t Full Known Flows Length: {len(known_test_flows)}")
    print(f"Partial Unknown Flows Length: {len(p_unknown_flows)} \t Partial Known Flows Length: {len(p_known_flows)}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_known_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
    print(f"Known Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Known Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_unknown_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
    print(f"Unknown Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Unknown Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}\n")

# Full flows
flows, known_confidences, unknown_confidences = get_flow_confidences(known_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
print(f"Known Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Known Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
flows, known_confidences, unknown_confidences = get_flow_confidences(mirror_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
print(f"Unknown Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Unknown Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")


# ALL UNKNOWN
print("\n\nALL UNKNOWN #############################################")
name = f'{N}_U_Tree'
# Load models
mirror_known_bins = np.load(f"Models/{name}_Known_Bin_Ls.npy")
mirror_unknown_bins = np.load(f"Models/{name}_Unknown_Bin_Ls.npy")
gbdt = load(f'Models/{name}')

# 25, 50, 75% of flows
p = [0.25, 0.5, 0.75]
for percentage in p:
    p_unknown_flows = get_partial_flows(unknown_test_flows, percentage)
    p_known_flows = get_partial_flows(known_test_flows, percentage)
    print(f"Flow Percentage: {percentage}")
    print(f"Full Unknown Flows Length: {len(unknown_test_flows)} \t Full Known Flows Length: {len(known_test_flows)}")
    print(f"Partial Unknown Flows Length: {len(p_unknown_flows)} \t Partial Known Flows Length: {len(p_known_flows)}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_known_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
    print(f"Known Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Known Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_unknown_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
    print(f"Unknown Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Unknown Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}\n")

# Full flows
flows, known_confidences, unknown_confidences = get_flow_confidences(known_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
print(f"Known Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Known Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
flows, known_confidences, unknown_confidences = get_flow_confidences(unknown_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
print(f"Unknown Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Unknown Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")

MIRROR UNKNOWN ############################################
Flow Percentage: 0.25
Full Unknown Flows Length: 417 	 Full Known Flows Length: 22
Partial Unknown Flows Length: 18 	 Partial Known Flows Length: 21
Known Strict 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 21
Known Maj. 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 21
Unknown Strict 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 18
Unknown Maj. 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 18

Flow Percentage: 0.5
Full Unknown Flows Length: 417 	 Full Known Flows Length: 22
Partial Unknown Flows Length: 18 	 Partial Known Flows Length: 21
Known Strict 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 21
Known Maj. 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 21
Unknown Strict 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 18
Unknown Maj. 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 18

Flow Percentage: 0.75
Full Unknown Flows Length: 417 	 Full Known Flows Length: 22
Partial Unknown Flows Length: 18 	

### Final Features 

In [35]:
############################# 25 PACKET SUBFLOWS - GBDT 
N = 25
unknown_test_flows, mirror_test_flows, known_test_flows = load_final_test_dicts(N)
con_thresh = 0.95

# MIRROR UNKNOWN
print("MIRROR UNKNOWN ############################################")
name = f'{N}_M'
# Load models
mirror_known_bins = np.load(f"Models/{name}_Known_Bin_Ls.npy")
mirror_unknown_bins = np.load(f"Models/{name}_Unknown_Bin_Ls.npy")
gbdt = load(f'Models/{name}_GBDT')

# 25, 50, 75% of flows
p = [0.25, 0.5, 0.75]
for percentage in p:
    p_unknown_flows = get_partial_flows(mirror_test_flows, percentage)
    p_known_flows = get_partial_flows(known_test_flows, percentage)
    print(f"Flow Percentage: {percentage}")
    print(f"Full Unknown Flows Length: {len(mirror_test_flows)} \t Full Known Flows Length: {len(known_test_flows)}")
    print(f"Partial Unknown Flows Length: {len(p_unknown_flows)} \t Partial Known Flows Length: {len(p_known_flows)}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_known_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
    print(f"Known Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Known Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_unknown_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
    print(f"Unknown Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Unknown Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}\n")

# Full flows
flows, known_confidences, unknown_confidences = get_flow_confidences(known_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
print(f"Known Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Known Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
flows, known_confidences, unknown_confidences = get_flow_confidences(mirror_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
print(f"Unknown Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Unknown Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")


# ALL UNKNOWN
print("\n\nALL UNKNOWN #############################################")
name = f'{N}_U'
# Load models
mirror_known_bins = np.load(f"Models/{name}_Known_Bin_Ls.npy")
mirror_unknown_bins = np.load(f"Models/{name}_Unknown_Bin_Ls.npy")
gbdt = load(f'Models/{name}_GBDT')

# 25, 50, 75% of flows
p = [0.25, 0.5, 0.75]
for percentage in p:
    p_unknown_flows = get_partial_flows(unknown_test_flows, percentage)
    p_known_flows = get_partial_flows(known_test_flows, percentage)
    print(f"Flow Percentage: {percentage}")
    print(f"Full Unknown Flows Length: {len(unknown_test_flows)} \t Full Known Flows Length: {len(known_test_flows)}")
    print(f"Partial Unknown Flows Length: {len(p_unknown_flows)} \t Partial Known Flows Length: {len(p_known_flows)}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_known_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
    print(f"Known Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Known Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_unknown_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
    print(f"Unknown Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Unknown Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}\n")

# Full flows
flows, known_confidences, unknown_confidences = get_flow_confidences(known_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
print(f"Known Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Known Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
flows, known_confidences, unknown_confidences = get_flow_confidences(unknown_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
print(f"Unknown Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Unknown Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")

MIRROR UNKNOWN ############################################
Flow Percentage: 0.25
Full Unknown Flows Length: 302 	 Full Known Flows Length: 31
Partial Unknown Flows Length: 287 	 Partial Known Flows Length: 30
Known Strict 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 30
Known Maj. 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 30
Unknown Strict 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 287
Unknown Maj. 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 287

Flow Percentage: 0.5
Full Unknown Flows Length: 302 	 Full Known Flows Length: 31
Partial Unknown Flows Length: 287 	 Partial Known Flows Length: 30
Known Strict 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 30
Known Maj. 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 30
Unknown Strict 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 287
Unknown Maj. 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 287

Flow Percentage: 0.75
Full Unknown Flows Length: 302 	 Full Known Flows Length: 31
Partial Unknown Flows Length

In [36]:
############################# 100 PACKET SUBFLOWS - GBDT 
N = 100
unknown_test_flows, mirror_test_flows, known_test_flows = load_test_dicts(N)
con_thresh = 0.95

# MIRROR UNKNOWN
print("MIRROR UNKNOWN ############################################")
name = f'{N}_M'
# Load models
mirror_known_bins = np.load(f"Models/{name}_Known_Bin_Ls.npy")
mirror_unknown_bins = np.load(f"Models/{name}_Unknown_Bin_Ls.npy")
gbdt = load(f'Models/{name}_GBDT')

# 25, 50, 75% of flows
p = [0.25, 0.5, 0.75]
for percentage in p:
    p_unknown_flows = get_partial_flows(mirror_test_flows, percentage)
    p_known_flows = get_partial_flows(known_test_flows, percentage)
    print(f"Flow Percentage: {percentage}")
    print(f"Full Unknown Flows Length: {len(mirror_test_flows)} \t Full Known Flows Length: {len(known_test_flows)}")
    print(f"Partial Unknown Flows Length: {len(p_unknown_flows)} \t Partial Known Flows Length: {len(p_known_flows)}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_known_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
    print(f"Known Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Known Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_unknown_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
    print(f"Unknown Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Unknown Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}\n")

# Full flows
flows, known_confidences, unknown_confidences = get_flow_confidences(known_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
print(f"Known Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Known Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
flows, known_confidences, unknown_confidences = get_flow_confidences(mirror_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
print(f"Unknown Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Unknown Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")


# ALL UNKNOWN
print("\n\nALL UNKNOWN #############################################")
name = f'{N}_U'
# Load models
mirror_known_bins = np.load(f"Models/{name}_Known_Bin_Ls.npy")
mirror_unknown_bins = np.load(f"Models/{name}_Unknown_Bin_Ls.npy")
gbdt = load(f'Models/{name}_GBDT')

# 25, 50, 75% of flows
p = [0.25, 0.5, 0.75]
for percentage in p:
    p_unknown_flows = get_partial_flows(unknown_test_flows, percentage)
    p_known_flows = get_partial_flows(known_test_flows, percentage)
    print(f"Flow Percentage: {percentage}")
    print(f"Full Unknown Flows Length: {len(unknown_test_flows)} \t Full Known Flows Length: {len(known_test_flows)}")
    print(f"Partial Unknown Flows Length: {len(p_unknown_flows)} \t Partial Known Flows Length: {len(p_known_flows)}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_known_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
    print(f"Known Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Known Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_unknown_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
    print(f"Unknown Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Unknown Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}\n")

# Full flows
flows, known_confidences, unknown_confidences = get_flow_confidences(known_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
print(f"Known Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Known Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
flows, known_confidences, unknown_confidences = get_flow_confidences(unknown_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
print(f"Unknown Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Unknown Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")

MIRROR UNKNOWN ############################################
Flow Percentage: 0.25
Full Unknown Flows Length: 326 	 Full Known Flows Length: 35
Partial Unknown Flows Length: 191 	 Partial Known Flows Length: 32
Known Strict 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 32
Known Maj. 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 32
Unknown Strict 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 191
Unknown Maj. 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 191

Flow Percentage: 0.5
Full Unknown Flows Length: 326 	 Full Known Flows Length: 35
Partial Unknown Flows Length: 191 	 Partial Known Flows Length: 32
Known Strict 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 32
Known Maj. 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 32
Unknown Strict 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 191
Unknown Maj. 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 191

Flow Percentage: 0.75
Full Unknown Flows Length: 326 	 Full Known Flows Length: 35
Partial Unknown Flows Length

In [37]:
############################# 1000 PACKET SUBFLOWS - GBDT 
N = 1000
unknown_test_flows, mirror_test_flows, known_test_flows = load_test_dicts(N)
con_thresh = 0.95

# MIRROR UNKNOWN
print("MIRROR UNKNOWN ############################################")
name = f'{N}_M'
# Load models
mirror_known_bins = np.load(f"Models/{name}_Known_Bin_Ls.npy")
mirror_unknown_bins = np.load(f"Models/{name}_Unknown_Bin_Ls.npy")
gbdt = load(f'Models/{name}_GBDT')

# 25, 50, 75% of flows
p = [0.25, 0.5, 0.75]
for percentage in p:
    p_unknown_flows = get_partial_flows(mirror_test_flows, percentage)
    p_known_flows = get_partial_flows(known_test_flows, percentage)
    print(f"Flow Percentage: {percentage}")
    print(f"Full Unknown Flows Length: {len(mirror_test_flows)} \t Full Known Flows Length: {len(known_test_flows)}")
    print(f"Partial Unknown Flows Length: {len(p_unknown_flows)} \t Partial Known Flows Length: {len(p_known_flows)}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_known_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
    print(f"Known Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Known Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_unknown_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
    print(f"Unknown Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Unknown Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}\n")

# Full flows
flows, known_confidences, unknown_confidences = get_flow_confidences(known_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
print(f"Known Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Known Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
flows, known_confidences, unknown_confidences = get_flow_confidences(mirror_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
print(f"Unknown Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Unknown Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")


# ALL UNKNOWN
print("\n\nALL UNKNOWN #############################################")
name = f'{N}_U'
# Load models
mirror_known_bins = np.load(f"Models/{name}_Known_Bin_Ls.npy")
mirror_unknown_bins = np.load(f"Models/{name}_Unknown_Bin_Ls.npy")
gbdt = load(f'Models/{name}_GBDT')

# 25, 50, 75% of flows
p = [0.25, 0.5, 0.75]
for percentage in p:
    p_unknown_flows = get_partial_flows(unknown_test_flows, percentage)
    p_known_flows = get_partial_flows(known_test_flows, percentage)
    print(f"Flow Percentage: {percentage}")
    print(f"Full Unknown Flows Length: {len(unknown_test_flows)} \t Full Known Flows Length: {len(known_test_flows)}")
    print(f"Partial Unknown Flows Length: {len(p_unknown_flows)} \t Partial Known Flows Length: {len(p_known_flows)}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_known_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
    print(f"Known Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Known Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_unknown_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
    print(f"Unknown Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Unknown Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}\n")

# Full flows
flows, known_confidences, unknown_confidences = get_flow_confidences(known_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
print(f"Known Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Known Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
flows, known_confidences, unknown_confidences = get_flow_confidences(unknown_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
print(f"Unknown Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Unknown Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")

MIRROR UNKNOWN ############################################
Flow Percentage: 0.25
Full Unknown Flows Length: 427 	 Full Known Flows Length: 33
Partial Unknown Flows Length: 21 	 Partial Known Flows Length: 33
Known Strict 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 33
Known Maj. 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 33
Unknown Strict 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 21
Unknown Maj. 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 21

Flow Percentage: 0.5
Full Unknown Flows Length: 427 	 Full Known Flows Length: 33
Partial Unknown Flows Length: 21 	 Partial Known Flows Length: 33
Known Strict 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 33
Known Maj. 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 33
Unknown Strict 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 21
Unknown Maj. 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 21

Flow Percentage: 0.75
Full Unknown Flows Length: 427 	 Full Known Flows Length: 33
Partial Unknown Flows Length: 21 	

In [3]:
############################# 25 PACKET SUBFLOWS - TREE NAIVE FEATURES
N = 25
unknown_test_flows, mirror_test_flows, known_test_flows = load_naive_test_dicts(N) 
con_thresh = 0.95

# MIRROR UNKNOWN
print("MIRROR UNKNOWN ############################################")
name = f'{N}_M_Tree'
# Load models
mirror_known_bins = np.load(f"Models/{name}_Known_Bin_Ls.npy")
mirror_unknown_bins = np.load(f"Models/{name}_Unknown_Bin_Ls.npy")
gbdt = load(f'Models/{name}')

# 25, 50, 75% of flows
p = [0.25, 0.5, 0.75]
for percentage in p:
    p_unknown_flows = get_partial_flows(mirror_test_flows, percentage)
    p_known_flows = get_partial_flows(known_test_flows, percentage)
    print(f"Flow Percentage: {percentage}")
    print(f"Full Unknown Flows Length: {len(mirror_test_flows)} \t Full Known Flows Length: {len(known_test_flows)}")
    print(f"Partial Unknown Flows Length: {len(p_unknown_flows)} \t Partial Known Flows Length: {len(p_known_flows)}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_known_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
    print(f"Known Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Known Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_unknown_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
    print(f"Unknown Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Unknown Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}\n")

# Full flows
flows, known_confidences, unknown_confidences = get_flow_confidences(known_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
print(f"Known Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Known Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
flows, known_confidences, unknown_confidences = get_flow_confidences(mirror_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
print(f"Unknown Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Unknown Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")


# ALL UNKNOWN
print("\n\nALL UNKNOWN #############################################")
name = f'{N}_U_Tree'
# Load models
mirror_known_bins = np.load(f"Models/{name}_Known_Bin_Ls.npy")
mirror_unknown_bins = np.load(f"Models/{name}_Unknown_Bin_Ls.npy")
gbdt = load(f'Models/{name}')

# 25, 50, 75% of flows
p = [0.25, 0.5, 0.75]
for percentage in p:
    p_unknown_flows = get_partial_flows(unknown_test_flows, percentage)
    p_known_flows = get_partial_flows(known_test_flows, percentage)
    print(f"Flow Percentage: {percentage}")
    print(f"Full Unknown Flows Length: {len(unknown_test_flows)} \t Full Known Flows Length: {len(known_test_flows)}")
    print(f"Partial Unknown Flows Length: {len(p_unknown_flows)} \t Partial Known Flows Length: {len(p_known_flows)}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_known_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
    print(f"Known Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Known Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    flows, known_confidences, unknown_confidences = get_flow_confidences(p_unknown_flows, \
                                                    mirror_known_bins, mirror_unknown_bins, gbdt)
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
    print(f"Unknown Strict {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
    correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
    print(f"Unknown Maj. {percentage} Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}\n")

# Full flows
flows, known_confidences, unknown_confidences = get_flow_confidences(known_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh)
print(f"Known Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(1, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Known Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
flows, known_confidences, unknown_confidences = get_flow_confidences(unknown_test_flows, \
                                                mirror_known_bins, mirror_unknown_bins, gbdt)
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh)
print(f"Unknown Strict 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")
correct, uncertain, acc = classify_confidences(0, known_confidences, unknown_confidences, con_thresh, maj=True)
print(f"Unknown Maj. 1.00 Flows \t Acc: {acc} \t Uncertain: {uncertain} \t Correct: {correct}")

MIRROR UNKNOWN ############################################
Flow Percentage: 0.25
Full Unknown Flows Length: 348 	 Full Known Flows Length: 41
Partial Unknown Flows Length: 338 	 Partial Known Flows Length: 40
Known Strict 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 40
Known Maj. 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 40
Unknown Strict 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 338
Unknown Maj. 0.25 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 338

Flow Percentage: 0.5
Full Unknown Flows Length: 348 	 Full Known Flows Length: 41
Partial Unknown Flows Length: 338 	 Partial Known Flows Length: 40
Known Strict 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 40
Known Maj. 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 40
Unknown Strict 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 338
Unknown Maj. 0.5 Flows 	 Acc: 1.0 	 Uncertain: 0.0 	 Correct: 338

Flow Percentage: 0.75
Full Unknown Flows Length: 348 	 Full Known Flows Length: 41
Partial Unknown Flows Length